In [22]:
import pulp
from pulp import *
import pandas as pd
import numpy as np

In [4]:
n_warehouses = 2
n_customers = 4
cost_matrix = np.array([[1,3,0.5,4],[2.5,5,1.5,2.5]])
cost_matrix

array([[1. , 3. , 0.5, 4. ],
       [2.5, 5. , 1.5, 2.5]])

In [5]:
cust_demands = np.array([35000, 22000, 18000, 30000])
cust_demands

array([35000, 22000, 18000, 30000])

In [6]:
warehouse_supply = np.array([60000, 80000])

In [7]:
model = LpProblem("Supply-Demand-Problem", LpMinimize)

In [8]:
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1,n_warehouses+1)]
variable_names

['11', '21', '12', '22', '13', '23', '14', '24']

In [9]:
variable_names.sort()

In [10]:
DV_variables = LpVariable.matrix("X", variable_names , cat = "Integer" , lowBound= 0 )

In [11]:
allocation = np.array(DV_variables).reshape(2,4)

In [12]:
lpSum(allocation*cost_matrix)

1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0

In [13]:
obj_func = lpSum(allocation*cost_matrix)

In [14]:
model +=  obj_func
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

In [15]:
# Warehouse Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)

X_11 + X_12 + X_13 + X_14 <= 60000
X_21 + X_22 + X_23 + X_24 <= 80000


In [16]:
# Customer Constraints
for j in range(n_customers):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)

X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000


In [17]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
SUBJECT TO
Supply_Constraints_0: X_11 + X_12 + X_13 + X_14 <= 60000

Supply_Constraints_1: X_21 + X_22 + X_23 + X_24 <= 80000

Demand_Constraints_0: X_11 + X_21 >= 35000

Demand_Constraints_1: X_12 + X_22 >= 22000

Demand_Constraints_2: X_13 + X_23 >= 18000

Demand_Constraints_3: X_14 + X_24 >= 30000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

In [18]:
model.writeLP("Supply_demand_prob.lp")

[X_11, X_12, X_13, X_14, X_21, X_22, X_23, X_24]

In [19]:
model.solve()
# solvers.PULP_CBC_CMD(fracGap=0)
status =  LpStatus[model.status]
print(status)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kailashthiyagarajan/opt/anaconda3/envs/opt_env/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/wl/hpzsvb551856hqjkw0rzm8s00000gn/T/3c57d0d89c4044a0adb3b36a495659e4-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/wl/hpzsvb551856hqjkw0rzm8s00000gn/T/3c57d0d89c4044a0adb3b36a495659e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 52 RHS
At line 59 BOUNDS
At line 68 ENDATA
Problem MODEL has 6 rows, 8 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 200000 - 0.00 seconds
Cgl0004I processed model has 6 rows, 8 columns (8 integer (0 of which binary)) and 16 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of 200000 found by DiveCoefficient after 0 iterations and 0 nod

In [20]:
print("Total Cost:", model.objective.value())
# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 200000.0
X_11 = 35000.0
X_12 = 22000.0
X_13 = 3000.0
X_14 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 15000.0
X_24 = 30000.0


In [21]:
# Warehouse 1 and Warehouse 2 required capacity
for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Warehouse  1
60000.0
Warehouse  2
45000.0
